In [0]:
import pandas as pd


df = pd.read_csv('drive/Colab Notebooks/raw-data.csv',encoding = "ISO-8859-1")
df = df.drop(['title','citeulike.id'], axis=1)
df = df.dropna().reset_index(drop=True)
df.head()

doc.id ta sẽ dùng để kết nối với bảnh user-doc
ở bảng này ta sẽ dùng các thuộc tính abstract và title 

In [0]:
import re  # For preprocessing
import spacy 

nlp = spacy.load('en', disable=['ner', 'parser'])

def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [0]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['raw.abstract'])

In [0]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]


In [0]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()

In [0]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in df_clean['clean']]
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences =bigram[sent]

In [0]:
import multiprocessing

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
w2v_model.build_vocab(sentences, progress_per=10000)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(27043981, 52082220)

In [0]:

w2v_model.save("drive/Colab Notebooks/word2vec.model")
w2v_model.save("drive/Colab Notebooks/model.bin")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
